# ERDRI CDS
The "Set of common data elements for Rare Diseases Registration" is the first practical instrument released by the EU RD Platform aiming at increasing interoperability of RD registries.

It contains 16 data elements to be registered by each rare disease registry across Europe, which are considered to be essential for further research. They refer to patient's personal data, diagnosis, disease history and care pathway, information for research purposes and about disability.

The "Set of common data elements for Rare Diseases Registration" was produced by a Working Group coordinated by the JRC and composed of experts from EU projects which worked on common data sets: EUCERD Joint Action, EPIRARE and RD-Connect.

[Source](https://eu-rd-platform.jrc.ec.europa.eu/set-of-common-data-elements_en)

## 1. Defining the ERDRI CDS Data Model
Instead of loading in a data model from a file, we can also create a data model by defining it in code. This skips the parsing step and allows us to define the data model in a more flexible way.

![image](../res/imgs/notebooks/erdri_cds1.png)
![image](../res/imgs/notebooks/erdri_cds2.png)

## 1.1. Resources

Every data model in the modern digital health environment will probably rely on at least one or more code systems. Code systems might include terminologies, classifications, or ontologies. Popular code systems include LOINC, SNOMED CT, ICD-10, and Orphanet Rare Disease Ontology (ORDO).

In this case, we will use the following code systems (in order as listed in the CODING column of the ERDRI CDS data model above):
- [ORDO](https://www.orpha.net/en/disease): Orphanet Rare Disease Ontology
- [Alpha-ID-SE](https://www.bfarm.de/EN/Code-systems/Terminologies/Alpha-ID-SE/_node.html): Simplified, uniform and standardised coding of rare diseases according to ICD-10-GM
- [ICD-9](https://iris.who.int/handle/10665/39473): International Classification of Diseases, Ninth Revision
- [ICD-9-CM](https://archive.cdc.gov/www_cdc_gov/nchs/icd/icd9cm.htm#:~:text=ICD%2D9%2DCM%20is%20the,10%20for%20mortality%20coding%20started.): International Classification of Diseases, Ninth Revision, Clinical Modification
- [ICD-10](https://www.who.int/classifications/icd/en/): International Classification of Diseases, Tenth Revision
- [HGVS](https://hgvs.org/): Human Genome Variation Society
- [HGNC](https://www.genenames.org/): HUGO Gene Nomenclature Committee
- [OMIM](https://www.omim.org/): Online Mendelian Inheritance in Man
- [HPO](https://hpo.jax.org/): Human Phenotype Ontology


There are already some popular code systems predefined in the `phenopacket-mapper` package. We can use them directly in our data model. Although optional, it is recommended to add the correct or most recent version to each code system. This will help to ensure that the code system is correctly identified and used in the data model.

The versions added below are the most recent at the time of writing this notebook. You can check for the most recent version of each code system on the respective website.

In [ ]:
from phenopacket_mapper.data_standards.code_system import ORDO, ICD9, HGVS, HGNC, OMIM, HPO 

In [ ]:
resources = [
    ORDO.set_version("1.0.19 (2024-08-02)"),
    ICD9,
    HGVS.set_version("21.0.4 (2024-08-15)"),
    HGNC.set_version("2024-08-23"),
    OMIM.set_version("2024-09-12"),
    HPO.set_version("2024-06-07")
]

The keen eyed among you might have spotted that we forgot to add the ICD-9-CM, ICD-10, and Alpha-ID-SE code systems. We can add them to the list of resources by creating a new `CodeSystem` object for each of them. We can then add them to the `resources` list.:

In [ ]:
from phenopacket_mapper.data_standards.code_system import CodeSystem

In [ ]:
alpha = CodeSystem(name='Alpha-ID-SE', namespace_prefix='alpha', url='https://www.bfarm.de/EN/Code-systems/Terminologies/Alpha-ID-SE/_node.html')
icd9cm = CodeSystem(name='International Classification of Diseases 9 Clinical Modification (USA)', namespace_prefix='icd-9-cm', url='http://hl7.org/fhir/sid/icd-9-cm')
icd10 = CodeSystem(name='International Classification of Diseases 10 (WHO)', namespace_prefix='icd-10', url='http://hl7.org/fhir/sid/icd-10')